<a href="https://colab.research.google.com/github/seonae0223/Deep_Learning/blob/main/06_%EC%A0%95%EA%B7%9C%ED%99%94(_Regularization_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST 데이터 준비

In [ ]:
import torchvision
import torch

train_ds = torchvision.datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

test_ds = torchvision.datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

train_dataloader = torch.utils.data.DataLoader(
    train_ds, batch_size=64, num_workers=2, shuffle=True  # dataset
)

test_dataloader = torch.utils.data.DataLoader(
    test_ds,
    batch_size=64,
    num_workers=2,
    shuffle=False,  # we don't need to shuffle the test data
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4647638.59it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134492.85it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1093348.96it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4205414.74it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# 모델 정의

In [ ]:
from torch import nn


# 위 방법처럼도 구현할 수 있으나, 더 간단하게 nn.Sequential로 구현할 수 있다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Neural Network을 구성하는 layer들을
        # initialize하는 부분
        self.fc_layers = nn.Sequential(
            nn.Linear(784, 784 // 4),
            nn.ReLU(),
            nn.Linear(784 // 4, 784 // 16),
            nn.ReLU(),
            nn.Linear(784 // 16, 10),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # Neural Network의 forward pass을 정의하는 부분
        # x은 input tensor
        x = torch.flatten(x, start_dim=1)
        x = self.fc_layers(x)
        return x

# L1 Regularization

L1 Regularization은 다음과 같은 식으로 정의됩니다.
$$
\hat{L}(y, \hat{y}; W) = L(y, \hat{y}; W) + \alpha \| W \|_1
$$

여기서 $W$ 은 모델의 Weight parameter들입니다.\
따라서 L1 Regularization의 항을 계산하기 위해서는 Model의 Weight parameter 값들이 필요합니다.


In [ ]:
model = NeuralNetwork()

모델의 Weight parameter들은 ```model.parameters()``` 메서드로 불러올 수 있습니다.

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7c8b2be345f0>

In [ ]:
# L1 Loss에 대한 구현

def L1Loss(model, alpha=1e-5):
    l1_loss = sum([p.abs().sum() for p in model.parameters()])
    return l1_loss * alpha

In [ ]:
l1_loss = L1Loss(model)
print(l1_loss)

tensor(0.0313, grad_fn=<MulBackward0>)


In [ ]:
from tqdm import tqdm


# Training Loop에서 L1 Regularization 적용하기
def train_loop(model, train_dataloader, loss_fn, optimizer):
    train_loss_history = []
    tbar = tqdm(train_dataloader)
    model.train()

    for batch, label in tbar:
        pred = model(batch)
        loss = loss_fn(pred, label) + L1Loss(model, alpha=0.001)  # 기존의 Loss에 L1Loss를 더해줍니다.
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        tbar.set_description(f"Train Loss: {loss.item():.4f}")
        train_loss_history.append(loss.item())

    return train_loss_history

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
train_loss_history = train_loop(model, train_dataloader, loss_fn, optimizer)

Train Loss: 5.2783: 100%|██████████| 938/938 [00:15<00:00, 62.02it/s]


# L2 Regularization

L2 Regularization은 다음과 같은 식으로 정의됩니다.
$$
\hat{L}(y, \hat{y}; W) = L(y, \hat{y}; W) + \alpha \| W \|_2^2
$$


In [ ]:
# L2 Loss에 대한 구현

def L2Loss(model, alpha=1e-5):
    l2_loss = sum([p.pow(2.0).sum() for p in model.parameters()])
    return l2_loss * alpha

In [ ]:
l2_loss = L2Loss(model)
print(l2_loss)

tensor(0.0008, grad_fn=<MulBackward0>)


In [ ]:
# Training Loop에서 L2 Regularization 적용하기
def train_loop(model, train_dataloader, loss_fn, optimizer):
    train_loss_history = []
    tbar = tqdm(train_dataloader)
    model.train()

    for batch, label in tbar:
        pred = model(batch)
        batch.to('cuda')
        loss = loss_fn(pred, label) + L2Loss(model, alpha=0.001)  # 기존의 Loss에 L2Loss를 더해줍니다.
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        tbar.set_description(f"Train Loss: {loss.item():.4f}")
        train_loss_history.append(loss.item())

    return train_loss_history

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
train_loss_history = train_loop(model, train_dataloader, loss_fn, optimizer)

Train Loss: 2.3737: 100%|██████████| 938/938 [00:14<00:00, 64.18it/s]


# Weight Decay

Weight Decay은 다음과 같은 형태로 Weight을 Update합니다.

$$
W_t = W_{t-1} - \nabla_W L_{CE}  - \gamma \cdot \lambda \cdot W_{t-1}
$$

Weight Decay은 PyTorch에서 매우 쉽게 구현할 수 있고, `torch.optim.SGD`에서 제공하는 `weight_decay` 파라미터를 설정해주면 됩니다.



In [ ]:
weight_decay = 0.001
optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.01,
                            weight_decay=weight_decay)  # Weight decay은 이렇게 명시할 수 있다.


# Dropout

Dropout이란 뉴럴넷을 학습시킬때, p의 확률로 일부의 neuron들을 비활성화시키는 방법입니다. Dropout은 Neural Network(Linear)의 Layer 사이사이에 위치합니다. 보통 같이 Dropout을 각 Activation function 뒤에 각 Layer마다 적용해줍니다.


In [ ]:
import torch
from torch import nn


# 위 방법처럼도 구현할 수 있으나, 더 간단하게 nn.Sequential로 구현할 수 있다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Neural Network을 구성하는 layer들을
        # initialize하는 부분
        self.fc_layers = nn.Sequential(
            nn.Linear(784, 784 // 4),
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout
            nn.Linear(784 // 4, 784 // 16),
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout
            nn.Linear(784 // 16, 10),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # Neural Network의 forward pass을 정의하는 부분
        # x은 input tensor
        x = torch.flatten(x, start_dim=1)
        x = self.fc_layers(x)
        return x


In [ ]:
model_with_dropout = NeuralNetwork()

In [ ]:
model_with_dropout

NeuralNetwork(
  (fc_layers): Sequential(
    (0): Linear(in_features=784, out_features=196, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=196, out_features=49, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=49, out_features=10, bias=True)
    (7): Sigmoid()
  )
)

# Early Stopping

Early Stopping이란 Validation loss가 증가하기 시작할때 학습을 중단하는 방법입니다. Training, Validation loop에서 구현해보도록 하겠습니다.

In [ ]:
import os

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.001)
train_loss_history = []
test_loss_history = []
test_acc_history = []
epochs = 1000
best_acc = 0.
os.makedirs("checkpoints", exist_ok=True)

In [ ]:
class EarlyStopping:
    def __init__(self, n_patience=5) -> None:
        self.n_patience = n_patience
        self.loss_history = []

    def __call__(self, loss: float):
        """Early stopping call function

        Args:
            loss (float): test loss function

        Returns:
            bool: False if early stopping condition is not met, True otherwise.
        """
        if len(self.loss_history) < self.n_patience:
            early_stop_flag = False
        elif min(self.loss_history[-self.n_patience:]) <= loss:
            # Early stopping condition is met
            # Current loss value is higher than the lowest loss value in the last n_patience epochs.
            early_stop_flag = True
        else:
            early_stop_flag = False

        self.loss_history.append(loss)
        return early_stop_flag


In [ ]:
model.to('cuda')

def train_loop(model, train_dataloader, loss_fn, optimizer):
    device = next(model.parameters()).device

    train_loss_history = []
    tbar = tqdm(train_dataloader)
    model.train()

    for batch, label in tbar:
        batch = batch.to(device)
        label = label.to(device)

        pred = model(batch)
        loss = loss_fn(pred, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        tbar.set_description(f"Train Loss: {loss.item():.4f}")
        train_loss_history.append(loss.item())

    return train_loss_history

def val_loop(model, test_dataloader, loss_fn):
    device = next(model.parameters()).device

    tbar = tqdm(test_dataloader)
    num_batches = len(test_dataloader)
    size = len(test_dataloader.dataset)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch, label in tbar:
            batch = batch.to(device)
            label = label.to(device)

            pred = model(batch)
            loss = loss_fn(pred, label)
            test_loss += loss.item()
            correct += (pred.argmax(1) == label).type(torch.float).sum().item()

            tbar.set_description(f"Test Loss: {loss.item():.4f}")
    test_loss /= num_batches
    correct /= size

    return test_loss, correct

In [ ]:
# 3회 에폭 동안 loss가 개선되지 않으면 중단.
n_patience = 3
early_stopping = EarlyStopping(n_patience=n_patience)

for epoch in range(epochs):
    print(f"------------------Epoch {epoch} Train------------------")
    train_losses = train_loop(model, train_dataloader, loss_fn, optimizer)
    train_loss_history.extend(train_losses)
    print(f"------------------Epoch {epoch} Test------------------")
    test_loss, test_acc = val_loop(model, test_dataloader, loss_fn)
    test_loss_history.append(test_loss)
    test_acc_history.append(test_acc)
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "checkpoints/best_model.pt")
    early_stop_flag = early_stopping(test_loss)
    if early_stop_flag:
        print("Early Stopping!")
        break



------------------Epoch 0 Train------------------


Train Loss: 2.2965: 100%|██████████| 938/938 [00:13<00:00, 67.00it/s]


------------------Epoch 0 Test------------------


Test Loss: 2.2937: 100%|██████████| 157/157 [00:02<00:00, 76.34it/s]


------------------Epoch 1 Train------------------


...

Train Loss: 1.5589: 100%|██████████| 938/938 [00:17<00:00, 53.52it/s]


------------------Epoch 866 Test------------------


Test Loss: 1.5123: 100%|██████████| 157/157 [00:02<00:00, 56.22it/s]


------------------Epoch 867 Train------------------


Train Loss: 1.5502:  55%|█████▌    | 518/938 [00:09<00:06, 60.62it/s]

In [ ]:
import os
os.makedirs("figures", exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(train_loss_history)
plt.title('Train loss history')
plt.savefig('figures/train_loss_history.png')

In [ ]:
plt.figure()
plt.plot(test_loss_history)
plt.title('Test loss history')
plt.savefig('figures/test_loss_history.png')

In [ ]:
plt.figure()
plt.plot(test_acc_history)
plt.title('Test accuracy history')
plt.savefig('figures/test_acc_history.png')